In [ ]:
!pip install datasets transformers evaluate rouge_score torch accelerate langchain-google-genai python-dotenv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.3 MB/s eta 0:00:00
   ━━━

In [ ]:
!pip install -U datasets huggingface_hub fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
import os
import time
# from dotenv import load_dotenv # No longer needed if using Colab Secrets
import google.generativeai as genai
import datasets
import evaluate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata # Import userdata

def evaluate_gemini_flash():
    """
    Google Gemini 모델들의 뉴스 요약 성능을 평가하고 ROUGE 점수를 비교합니다.
    """
    # 1. API 키 설정 및 환경 준비
    print("🔑 Google API 키를 로드합니다...")
    # Load API key from Colab Secrets
    google_api_key = userdata.get('GOOGLE_API_KEY')

    if not google_api_key:
        print("API 키가 설정되지 않았습니다. Colab의 Secrets Manager에 'GOOGLE_API_KEY'를 설정해주세요.")
        return

    try:
        genai.configure(api_key=google_api_key)
    except Exception as e:
        print(f"API 키 설정에 실패했습니다. 오류: {e}")
        return

    # The rest of the code remains the same...
    # 2. 데이터셋 및 평가 지표 로드
    print("📦 데이터셋과 ROUGE 평가 지표를 로드합니다...")
    news_dataset = datasets.load_dataset("daekeun-ml/naver-news-summarization-ko", split="test")
    rouge = evaluate.load('rouge')

    # 🚨 중요: API 호출 비용/사용량 관리를 위해 샘플 수를 조절하세요.
    SAMPLE_SIZE = 50
    test_samples = news_dataset.select(range(SAMPLE_SIZE))

    # 3. Gemini 모델 정의
    model_name = "gemini-1.5-flash-latest"

    results = {}
    print(f"🚀 Gemini 모델 평가를 시작합니다. (샘플 개수: {SAMPLE_SIZE}개)")

    # 4. 요약을 위한 프롬프트 템플릿 정의
    prompt_template = """
    당신은 편견 없이 사실만을 전달하는 전문 뉴스 에디터입니다.
    아래 뉴스 기사의 핵심 내용을 원문에 기반하여 3~4개의 문장으로 간결하게 요약해 주세요.
    당신의 의견, 추론, 또는 원문에 없는 정보는 절대 포함하지 마세요.

    ---
    [뉴스 기사]
    {article_text}
    ---

    [요약 결과]
    """
    prompt = ChatPromptTemplate.from_template(prompt_template)
    output_parser = StrOutputParser()

    # 5. 모델별 평가 루프
    print(f"\n--- 모델 평가 중: {model_name} ---")
    try:
        # LangChain을 통해 Gemini 모델 초기화
        # Pass google_api_key explicitly to ChatGoogleGenerativeAI
        llm = ChatGoogleGenerativeAI(model=model_name, google_api_key=google_api_key, temperature=0.1)
        chain = prompt | llm | output_parser
    except Exception as e:
        print(f"모델 '{model_name}' 로딩 실패: {e}\n")

    predictions = []
    references = []
    for i, sample in enumerate(test_samples):
        input_text = f"제목: {sample['title']}\n\n본문: {sample['document']}"
        try:
            # API 호출로 요약문 생성
            generated_summary = chain.invoke({"article_text": input_text})
            predictions.append(generated_summary)
            references.append(sample['summary'])
            print(f"\r  - 진행률: {i+1}/{SAMPLE_SIZE}", end="")
            # API 과호출 방지를 위한 딜레이 (초당 쿼리 수 제한이 있을 수 있음)
            time.sleep(10)
        except Exception as e:
            print(f"\n샘플 {i} 처리 중 API 오류 발생: {e}")
            predictions.append("") # 오류 시 빈 문자열로 처리
            references.append(sample['summary'])
    print("\n  - ROUGE 점수 계산 중...")
    score = rouge.compute(predictions=predictions, references=references)
    results[model_name] = {k: round(v * 100, 2) for k, v in score.items()}
    print(f"  - 계산 완료!")

    # 6. 최종 결과 출력
    print("\n--- 📊 최종 평가 결과 (Gemini) ---")
    print(f"{'모델 이름':<30} | {'ROUGE-1':<10} | {'ROUGE-2':<10} | {'ROUGE-L':<10}")
    print("-" * 70)
    for model_name, scores in results.items():
        r1 = scores.get('rouge1', 'N/A')
        r2 = scores.get('rouge2', 'N/A')
        rl = scores.get('rougeL', 'N/A')
        print(f"{model_name:<30} | {r1:<10} | {r2:<10} | {rl:<10}")

    print("\n✅ 평가가 완료되었습니다.")

In [ ]:
if __name__ == '__main__':
    evaluate_gemini_flash()

🔑 Google API 키를 로드합니다...
📦 데이터셋과 ROUGE 평가 지표를 로드합니다...
🚀 Gemini 모델 평가를 시작합니다. (샘플 개수: 50개)

--- 모델 평가 중: gemini-1.5-flash-latest ---
  - 진행률: 50/50
  - ROUGE 점수 계산 중...
  - 계산 완료!

--- 모델 평가 중: gemini-1.5-pro-latest ---


  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/


샘플 0 처리 중 API 오류 발생: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensio

  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/


샘플 1 처리 중 API 오류 발생: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensio

...

  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/


샘플 13 처리 중 API 오류 발생: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensi

  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/


샘플 14 처리 중 API 오류 발생: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensi

KeyboardInterrupt: 

In [ ]:
def evaluate_gemini_pro():
    """
    Google Gemini 모델들의 뉴스 요약 성능을 평가하고 ROUGE 점수를 비교합니다.
    """
    # 1. API 키 설정 및 환경 준비
    print("🔑 Google API 키를 로드합니다...")
    # Load API key from Colab Secrets
    google_api_key = userdata.get('GOOGLE_API_KEY')

    if not google_api_key:
        print("API 키가 설정되지 않았습니다. Colab의 Secrets Manager에 'GOOGLE_API_KEY'를 설정해주세요.")
        return

    try:
        genai.configure(api_key=google_api_key)
    except Exception as e:
        print(f"API 키 설정에 실패했습니다. 오류: {e}")
        return

    # The rest of the code remains the same...
    # 2. 데이터셋 및 평가 지표 로드
    print("📦 데이터셋과 ROUGE 평가 지표를 로드합니다...")
    news_dataset = datasets.load_dataset("daekeun-ml/naver-news-summarization-ko", split="test")
    rouge = evaluate.load('rouge')

    # 🚨 중요: API 호출 비용/사용량 관리를 위해 샘플 수를 조절하세요.
    SAMPLE_SIZE = 50
    test_samples = news_dataset.select(range(SAMPLE_SIZE))

    # 3. Gemini 모델 정의
    model_name = "gemini-1.5-pro-latest"

    results = {}
    print(f"🚀 Gemini 모델 평가를 시작합니다. (샘플 개수: {SAMPLE_SIZE}개)")

    # 4. 요약을 위한 프롬프트 템플릿 정의
    prompt_template = """
    당신은 편견 없이 사실만을 전달하는 전문 뉴스 에디터입니다.
    아래 뉴스 기사의 핵심 내용을 원문에 기반하여 3~4개의 문장으로 간결하게 요약해 주세요.
    당신의 의견, 추론, 또는 원문에 없는 정보는 절대 포함하지 마세요.

    ---
    [뉴스 기사]
    {article_text}
    ---

    [요약 결과]
    """
    prompt = ChatPromptTemplate.from_template(prompt_template)
    output_parser = StrOutputParser()

    # 5. 모델별 평가 루프
    print(f"\n--- 모델 평가 중: {model_name} ---")
    try:
        # LangChain을 통해 Gemini 모델 초기화
        # Pass google_api_key explicitly to ChatGoogleGenerativeAI
        llm = ChatGoogleGenerativeAI(model=model_name, google_api_key=google_api_key, temperature=0.1)
        chain = prompt | llm | output_parser
    except Exception as e:
        print(f"모델 '{model_name}' 로딩 실패: {e}\n")

    predictions = []
    references = []
    for i, sample in enumerate(test_samples):
        input_text = f"제목: {sample['title']}\n\n본문: {sample['document']}"
        try:
            # API 호출로 요약문 생성
            generated_summary = chain.invoke({"article_text": input_text})
            predictions.append(generated_summary)
            references.append(sample['summary'])
            print(f"\r  - 진행률: {i+1}/{SAMPLE_SIZE}", end="")
            # API 과호출 방지를 위한 딜레이 (초당 쿼리 수 제한이 있을 수 있음)
            time.sleep(60)
        except Exception as e:
            print(f"\n샘플 {i} 처리 중 API 오류 발생: {e}")
            predictions.append("") # 오류 시 빈 문자열로 처리
            references.append(sample['summary'])
    print("\n  - ROUGE 점수 계산 중...")
    score = rouge.compute(predictions=predictions, references=references)
    results[model_name] = {k: round(v * 100, 2) for k, v in score.items()}
    print(f"  - 계산 완료!")

    # 6. 최종 결과 출력
    print("\n--- 📊 최종 평가 결과 (Gemini) ---")
    print(f"{'모델 이름':<30} | {'ROUGE-1':<10} | {'ROUGE-2':<10} | {'ROUGE-L':<10}")
    print("-" * 70)
    for model_name, scores in results.items():
        r1 = scores.get('rouge1', 'N/A')
        r2 = scores.get('rouge2', 'N/A')
        rl = scores.get('rougeL', 'N/A')
        print(f"{model_name:<30} | {r1:<10} | {r2:<10} | {rl:<10}")

    print("\n✅ 평가가 완료되었습니다.")

In [ ]:
evaluate_gemini_pro()

🔑 Google API 키를 로드합니다...
📦 데이터셋과 ROUGE 평가 지표를 로드합니다...


  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/

🚀 Gemini 모델 평가를 시작합니다. (샘플 개수: 50개)

--- 모델 평가 중: gemini-1.5-pro-latest ---


  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/


샘플 0 처리 중 API 오류 발생: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensio

...

  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/


샘플 2 처리 중 API 오류 발생: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensio

  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/


샘플 3 처리 중 API 오류 발생: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensio

KeyboardInterrupt: 

In [ ]:
import torch
import datasets
import evaluate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def evaluate_hf_t5_model():
    """
    HuggingFace의 'Xcz2568/T5-summarization-Korean1' 모델에 대해
    뉴스 요약 성능을 평가하고 ROUGE 점수를 계산합니다.
    """
    # 1. 모델 이름 및 환경 설정
    model_name = "Xcz2568/T5-summarization-Korean1"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"🚀 HuggingFace 모델 평가를 시작합니다.")
    print(f"- 모델: {model_name}")
    print(f"- 장치: {device}")

    # 2. 데이터셋 및 평가 지표 로드
    print("📦 데이터셋과 ROUGE 평가 지표를 로드합니다...")
    news_dataset = datasets.load_dataset("daekeun-ml/naver-news-summarization-ko", split="test")
    rouge = evaluate.load('rouge')

    # 평가할 샘플 수 지정
    SAMPLE_SIZE = 50
    test_samples = news_dataset.select(range(SAMPLE_SIZE))
    print(f"⚠️ 테스트 샘플 개수: {SAMPLE_SIZE}개\n")

    # 3. 모델 및 토크나이저 로드
    print(f"--- 모델 로딩 중: {model_name} ---")
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
        print("✅ 모델 로딩 완료!")
    except Exception as e:
        print(f"모델 로딩 실패: {e}\n")
        return

    # 4. 모델 평가 루프
    predictions = []
    references = []

    for i, sample in enumerate(test_samples):
        # T5 모델은 입력 텍스트 앞에 'summarize: '를 붙여주는 것이 성능에 좋습니다.
        input_text = f"summarize: {sample['title']}. {sample['document']}"

        try:
            # 이전 오류들을 해결한 안정적인 요약 생성 방식
            # 1. 입력 텍스트를 최대 길이에 맞춰 토큰화 및 절단
            inputs = tokenizer(
                input_text,
                return_tensors="pt",
                max_length=1024, # T5 모델에 안전한 최대 길이로 고정
                truncation=True
            ).to(device)

            # 2. 요약 토큰 생성
            summary_ids = model.generate(
                inputs['input_ids'],
                max_length=128,
                min_length=32,
                num_beams=4, # Beam search로 더 나은 품질의 결과 생성
                early_stopping=True
            )

            # 3. 생성된 토큰을 다시 텍스트로 변환
            generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

            predictions.append(generated_summary)
            references.append(sample['summary'])

            print(f"\r  - 진행률: {i+1}/{SAMPLE_SIZE}", end="")

        except Exception as e:
            print(f"\n샘플 {i} 처리 중 오류 발생: {e}")
            predictions.append("")
            references.append(sample['summary'])

    # 5. ROUGE 점수 계산 및 결과 출력
    print("\n  - ROUGE 점수 계산 중...")
    score = rouge.compute(predictions=predictions, references=references)
    final_result = {k: round(v * 100, 2) for k, v in score.items()}
    print(f"  - 계산 완료!\n")

    print(f"--- 📊 최종 평가 결과 ({model_name}) ---")
    print(f"{'지표':<10} | {'점수':<10}")
    print("-" * 25)
    print(f"{'ROUGE-1':<10} | {final_result.get('rouge1', 'N/A'):<10}")
    print(f"{'ROUGE-2':<10} | {final_result.get('rouge2', 'N/A'):<10}")
    print(f"{'ROUGE-L':<10} | {final_result.get('rougeL', 'N/A'):<10}")

    print("\n✅ 평가가 완료되었습니다.")

In [ ]:
if __name__ == '__main__':
    evaluate_hf_t5_model()

🚀 HuggingFace 모델 평가를 시작합니다.
- 모델: Xcz2568/T5-summarization-Korean1
- 장치: cpu
📦 데이터셋과 ROUGE 평가 지표를 로드합니다...
⚠️ 테스트 샘플 개수: 50개

--- 모델 로딩 중: Xcz2568/T5-summarization-Korean1 ---


tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

✅ 모델 로딩 완료!
  - 진행률: 50/50
  - ROUGE 점수 계산 중...
  - 계산 완료!

--- 📊 최종 평가 결과 (Xcz2568/T5-summarization-Korean1) ---
지표         | 점수        
-------------------------
ROUGE-1    | 22.1      
ROUGE-2    | 9.25      
ROUGE-L    | 21.3      

✅ 평가가 완료되었습니다.
